In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import matplotlib.pyplot as plt 
import pandas as pd
import time
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import sys
sys.path.append("../camp_scripts") # change accordingly
sns.set_context('notebook')

In [4]:
from june.demography.geography import Geography
from june.demography.demography import load_age_and_sex_generators_for_bins, Demography, Population
from june.paths import data_path
from june.infection_seed import InfectionSeed
from june.simulator import Simulator
from june.infection import InfectionSelector
from june.interaction import Interaction
from june.groups import Hospital, Hospitals
from june.distributors import HospitalDistributor
from june.hdf5_savers import generate_world_from_hdf5
from june.groups import Cemeteries
from june.policy import Policy, Policies
from june.logger.read_logger import ReadLogger

from camps.paths import camp_data_path
from camps.world import World
from camps.groups.leisure import generate_leisure_for_world, generate_leisure_for_config
from camp_creation import generate_empty_world, populate_world, distribute_people_to_households # this is loaded from the ../camp_scripts folder

No --data argument given - defaulting to:
/home/florpi/camps/JUNE/data
No --configs argument given - defaulting to:
/home/florpi/camps/JUNE/configs


2020-09-01 19:22:04,933 - camps.paths - WARNING - No --data argument given - defaulting to:
/home/florpi/camps/JUNE/data
2020-09-01 19:22:04,934 - camps.paths - WARNING - No --configs argument given - defaulting to:
/home/florpi/camps/JUNE/configs
2020-09-01 19:22:04,934 - camps.paths - WARNING - No --camp_data argument given - defaulting to:
/home/florpi/camps/JUNE/camp_data
2020-09-01 19:22:04,935 - camps.paths - WARNING - No --configs_camps argument given - defaulting to:
/home/florpi/camps/JUNE/configs_camps


ImportError: cannot import name 'generate_world_from_hdf5' from 'june.world' (/home/florpi/camps/JUNE/june/world.py)

In [ ]:
from camps.groups import PumpLatrines, PumpLatrineDistributor
from camps.groups import DistributionCenters, DistributionCenterDistributor
from camps.groups import Communals, CommunalDistributor
from camps.groups import FemaleCommunals, FemaleCommunalDistributor
from camps.groups import Religiouss, ReligiousDistributor
from camps.groups import Shelter, Shelters, ShelterDistributor
from camps.groups import IsolationUnit, IsolationUnits
from june.groups.leisure import HouseholdVisitsDistributor
from camps.groups import PlayGroups, PlayGroupDistributor

## Initialize the world

In [ ]:
# create empty world's geography
world = generate_empty_world(filter_key = {"region" : ["CXB-219"]})

In [ ]:
# populate empty world
populate_world(world)

In [ ]:
# distribute people to households
distribute_people_to_households(world)

In [ ]:
# this is not working
hospitals= Hospitals.from_file(
    filename=camp_data_path / 'input/hospitals/hospitals.csv'
)
world.hospitals = hospitals
for hospital in world.hospitals:
    hospital.super_area = world.super_areas.members[0]
hospital_distributor = HospitalDistributor(hospitals, 
                                           medic_min_age=20,
                                           patients_per_medic=10)

hospital_distributor.distribute_medics_from_world(world.people)
world.isolation_units = IsolationUnits([IsolationUnit()])

In [ ]:
world.pump_latrines = PumpLatrines.for_areas(world.areas)
world.distribution_centers = DistributionCenters.for_areas(world.areas)
world.communals = Communals.for_areas(world.areas)
world.female_communals = FemaleCommunals.for_areas(world.areas)
world.religiouss = Religiouss.for_areas(world.areas)
world.play_groups = PlayGroups.for_areas(world.areas)

In [ ]:
#world.box_mode = False
world.cemeteries = Cemeteries()

## Geography exploration

### UNHCR camp blocks inside the World

In [ ]:
super_areas = []
for super_area in world.super_areas:
    super_areas.append(super_area.name)

In [ ]:
super_areas[:5]

### Mahjee blocks inside World

In [ ]:
areas = []
for area in world.areas:
    areas.append(area.name)

In [ ]:
areas[:5]

## Demography exploration

### UNHCR block residents

In [ ]:
sample_ages = []
sample_sexes = []
for person in world.super_areas.members[0].people:
    sample_ages.append(person.age)
    sample_sexes.append(person.sex)

In [ ]:
sample_sexes[:5], sample_ages[:5]

In [ ]:
ages, counts = np.unique(sample_ages, return_counts=True)
plt.bar(ages, counts)
plt.title('super area: {} age distribution'.format(world.super_areas.members[0].name))
plt.show()

In [ ]:
print ('The % children between ages 0-16 is {}%'.format(round(np.sum(counts[:16])/(np.sum(counts[:16])+np.sum(counts[16:]))*100)))

## Households

### Household size distribution

In [ ]:
household_sizes = [household.size for household in world.households]
plt.hist(household_sizes, density=True, bins=np.arange(0,12))
plt.xlabel("Household size")
plt.ylabel("Frequency")
plt.axvline(np.mean(household_sizes), color="cyan")
plt.text(8, 0.150, f"Avg. size: {(np.mean(household_sizes)):.2f}")
plt.show()

# Shelters

In [ ]:
world.shelters = Shelters.for_areas(world.areas)
shelter_distributor = ShelterDistributor(sharing_shelter_ratio = 0.75) # proportion of families that share a shelter
for area in world.areas:
    shelter_distributor.distribute_people_in_shelters(area.shelters, area.households)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(8,3))
shelter_sizes = [shelter.size for shelter in world.shelters]
shelter_families = [shelter.n_families for shelter in world.shelters]
ax[0].hist(shelter_sizes)
ax[0].set_title("Shelter size")
shelter_families = {1 : len([shelter for shelter in world.shelters if shelter.n_families == 1]),
                    2 : len([shelter for shelter in world.shelters if shelter.n_families == 2])}
ax[1].bar(shelter_families.keys(), shelter_families.values())
ax[1].set_title("# families per shelter")
plt.show()

## Activity exploration

### Pumps and latrines 

In [ ]:
pumps_and_latrines = []
for pump_latrine in world.areas.members[0].pump_latrines:
    pumps_and_latrines.append(pump_latrine)

In [ ]:
print ('There are {} pumps and latrines in area {}'.format(len(pumps_and_latrines), world.areas.members[0].name))

### Distribution centers

In [ ]:
distribution_centers = []
for center in world.distribution_centers:
    distribution_centers.append(center)

In [ ]:
distribution_centers[0].coordinates

### Communal spaces

In [ ]:
communal_spaces = []
for space in world.communals:
    communal_spaces.append(space)

In [ ]:
communal_spaces[0].coordinates

### Female friendly communal spaces

In [ ]:
female_communal_spaces = []
for space in world.female_communals:
    female_communal_spaces.append(space)

In [ ]:
female_communal_spaces[0].coordinates

### Religious sites

In [ ]:
religious = []
for rel in world.religiouss:
    religious.append(rel)

In [ ]:
religious[0].coordinates

# Adding the infection

In [ ]:
selector = InfectionSelector.from_file()

# Adding the interaction

In [ ]:
interaction = Interaction.from_file(config_filename='../configs_camps/defaults/interaction/ContactInteraction.yaml')

# Set policies

In [ ]:
policies = Policies.from_file('../configs_camps/defaults/policy/home_care_policy.yaml', base_policy_modules=('june.policy', 'camps.policy'))

In [ ]:
policies.policies

# Seed the disease

In [ ]:
seed = InfectionSeed(world.super_areas,
           selector)

seed.unleash_virus(n_cases=5)

# Init leisure

In [ ]:
CONFIG_PATH = "../configs_camps/config_example.yaml"

leisure_instance = generate_leisure_for_config(
            world=world, config_filename=CONFIG_PATH
)
leisure_instance.leisure_distributors = {}
leisure_instance.leisure_distributors['pump_latrines'] = PumpLatrineDistributor.from_config(pump_latrines=world.pump_latrines)
leisure_instance.leisure_distributors['distribution_centers'] = DistributionCenterDistributor.from_config(distribution_centers=world.distribution_centers)
leisure_instance.leisure_distributors['communals'] = CommunalDistributor.from_config(communals=world.communals)
leisure_instance.leisure_distributors['female_communals'] = FemaleCommunalDistributor.from_config(female_communals=world.female_communals)
leisure_instance.leisure_distributors['play_groups'] = PlayGroupDistributor.from_config(play_groups=world.play_groups)

#associate social activities to shelters
leisure_instance.distribute_social_venues_to_households(world.shelters, world.super_areas)
leisure_instance.leisure_distributors

# Initialize simulator and run

In [ ]:
simulator = Simulator.from_file(
    world = world,
    interaction = interaction,
    infection_selector=selector,
    leisure = leisure_instance,
    policies=policies,
    config_filename = CONFIG_PATH
)

In [ ]:
%%time
simulator.run()

# Getting the results

All results are stored in a json file specified in the ``save_path`` variable in the config file. We can also access it from ``world.logger`` directly.

In [ ]:
import pandas as pd
read = ReadLogger()

In [ ]:
read.load_infection_location()

In [ ]:
infection_locations = read.get_locations_infections()

In [ ]:
unique_locations, counts_locations = np.unique(
                np.array(infection_locations),
                return_counts=True)

location_counts_df = pd.DataFrame(
    {'locations': unique_locations,
    'counts': counts_locations}
)
location_counts_df.set_index('locations', inplace=True)


In [ ]:
location_counts_df['percent_infections']= 100*(location_counts_df['counts'])/location_counts_df.values.sum()

In [ ]:
import matplotlib.ticker as mtick

ax = location_counts_df['percent_infections'].sort_values().plot.bar()
ax.yaxis.set_major_formatter(mtick.PercentFormatter())
plt.ylabel('Percentage of infections at location')
plt.xlabel('location')

In [ ]:
for location in unique_locations:
    read.locations_df[str(location)] = read.locations_df.apply(
        lambda x: x.location.count(str(location)), axis=1
    )
read.locations_df.index = pd.to_datetime(read.locations_df.index)

In [ ]:
read.locations_df.iloc[:,3:].plot()

## Where did outbreaks happen?

In [ ]:
unique_id, count_id =  np.unique(
        np.array(read.locations_df['location_id'].sum()),
        return_counts=True
)

In [ ]:
unique_id[np.argsort(count_id)[-5:]]

In [ ]:
count_id[np.argsort(count_id)[-5:]]

In [ ]:
world_df = read.world_summary()
world_df.plot()
plt.legend()

In [ ]:
world_df['daily_infections'].plot()

In [ ]:
world_df['daily_infections'].iloc[390:]

## Output to save

In [ ]:
%%time
summary = read.run_summary(
    super_area_region_path=camp_data_path 
    / 'input/geography/area_super_area_region.csv'
)

In [ ]:
summary